# Getting Live Audio Research

Absolutely. We can do this with many libraries but there are two more popular and easy to handle `pyaudio.` and `sounddevice`. 


## 1. With `sounddevice`
Let's use the `sounddevice` library in Python to capture live audio and select a specific input device. 

1. **List Available Input Devices**: 

    The first step is to list all available input devices so you can choose the microphone you want to use.

    


In [ ]:
import sounddevice as sd

def print_sound_devices():
    print(sd.query_devices())

print_sound_devices()

This function will print out a list of all input and output devices, including microphones. Each device has an index that we can use to select it for recording.


2. **Select Specific Microphone and Record Audio**: 

    After choosing the microphone (let's assume the index is 1), you can record audio from that specific microphone as follows:


In [ ]:
fs = 44100  # Sample rate
duration = 10  # Duration in seconds

def record_audio(device_index):
    print("Recording...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=2, device=device_index)
    sd.wait()  # Wait for the recording to finish
    print("Recording Complete.")
    return recording

recording = record_audio(device_index=1)



 In this code, `device_index` is the index of the microphone you want to use, `fs` is the sample rate, and `duration` is the duration of the recording in seconds.



3. **Continuous/Streamed Audio Recording**: 

    In order to perform real-time predictions, you need to continuously capture audio. The `sounddevice.InputStream` can be used for this purpose:

In [ ]:
def audio_callback(indata, frames, time, status):
    # This is called for every chunk of audio, 'indata' is a NumPy array with the captured audio

with sd.InputStream(callback=audio_callback, device=device_index):
    sd.sleep(duration * 1000)  # This will capture audio for 'duration' seconds



In the `audio_callback` function, you can add your model's prediction code. Please note, you need to adjust this example to your specific use case, as the audio processing and prediction steps depend heavily on your model and task.

## 2. using `pyaudio`

another popular library for audio processing is pyaudio. Here is an example of how you can use it for your task:




1. **Listing Available Input Devices**:

    You can use `pyaudio`'s `get_device_info_by_index` function in combination with a loop to print out all available audio devices.






  

In [ ]:
import pyaudio

def print_sound_devices():
    p = pyaudio.PyAudio()

    info = p.get_host_api_info_by_index(0)
    numdevices = info.get('deviceCount')
    for i in range(0, numdevices):
        if (p.get_device_info_by_index(i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", p.get_device_info_by_index(i).get('name'))

print_sound_devices()



This function will print out a list of all input devices, including microphones. Each device has an index that you can use to select it for recording.

2. **Selecting Specific Microphone and Recording Audio**:

    After choosing the microphone (let's assume the index is 1), you can record audio from that specific microphone as follows:


In [ ]:
def record_audio(device_index):
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=44100, input=True, input_device_index=device_index, frames_per_buffer=1024)
    print("Recording...")
    frames = []
    for i in range(0, int(44100 / 1024 * duration)):
        data = stream.read(1024)
        frames.append(data)
    print("Recording Complete.")
    stream.stop_stream()
    stream.close()
    p.terminate()
    return frames

frames = record_audio(device_index=1)


In this code, `device_index` is the index of the microphone you want to use, `44100` is the sample rate, and `duration` is the duration of the recording in seconds.


3. **Continuous/Streamed Audio Recording**:

    If you want to record audio in a stream to process in real-time, you can use `pyaudio`'s callback mode:


In [ ]:
def audio_callback(in_data, frame_count, time_info, status):
    # This is called for every chunk of audio, 'in_data' is a byte array with the captured audio

stream = p.open(format=pyaudio.paInt16, channels=1, rate=44100, input=True, input_device_index=device_index, frames_per_buffer=1024, stream_callback=audio_callback)

stream.start_stream()
while stream.is_active():  # This will capture audio as long as the stream is active
    time.sleep(0.1)
stream.stop_stream()
stream.close()
p.terminate()


In the `audio_callback` function, you can add your model's prediction code. Again, you will need to adjust this example to your specific use case.